# Imports

In [1]:
import os
import pathlib

import numpy as np
import pandas as pd
import sqlalchemy as sa
import statsmodels.api as sm
import statsmodels.stats.api as sms
from dotenv import dotenv_values
from statsmodels.stats.power import tt_ind_solve_power
from tqdm import tqdm
from datetime import datetime, timedelta

import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
os.environ["OMP_NUM_THREADS"] = "8"
os.environ["MKL_NUM_THREADS"] = "8"
os.environ["OPENBLAS_NUM_THREADS"] = "8"

In [3]:
config = dotenv_values("/home/jovyan/.env")
def get_query_clickhouse(q: str) -> pd.DataFrame:
    ch_host = config['CH_HOST']
    ch_port = config['CH_PORT']
    ch_db   = config['CH_READ_DB']
    ch_user = config['CH_READ_USER']
    ch_pass = config['CH_READ_PASS']

    engine = sa.create_engine(
        f"clickhouse+native://{ch_user}:"
        f"{ch_pass}@{ch_host}:"
        f"{ch_port}/{ch_db}?secure=True"
    )
    return pd.read_sql_query(q, con=engine)

# Functions

In [4]:
def get_MDE(metric, exp_ios_data, obs_ios_data, total_frac=1, test_fraction=0.5, alpha=0.05, power=0.8, optional=False):   
#     print(f"Целевая метрика - {metric}")

    alpha = alpha # alpha = 0.05, для которой считаем MDE
    power = power # мощность = 0.8, для которой считаем MDE
    test_fraction = test_fraction # Тест-контроль 50 на 50
    exp_gmv_data = exp_ios_data.copy()
    obs_gmv_data = obs_ios_data.copy().rename({metric: "CUPED_X"}, axis=1)

    gmv_data = exp_gmv_data[["anonymous_id", metric]].merge(obs_gmv_data[["anonymous_id", "CUPED_X"]], how="left", on="anonymous_id")
    gmv_data = gmv_data.sample(frac=total_frac)

    # обработаем юзеров без истории
    gmv_data["missing_CUPED"] = 0
    gmv_data.loc[gmv_data["CUPED_X"].isna(), "missing_CUPED"] = 1
    gmv_data = gmv_data.fillna(0)
    if optional:
        print(f"Пользователей без истории {gmv_data.missing_CUPED.mean():.4%}")
    # в соответствие с процедурой CUPED'a случайным образом выделим тестовую и контрольную группы

    gmv_data["treatment"] = np.random.choice([0,1], size=gmv_data.shape[0])
    # применим CUPED, получив по итогу пост-cuped метрику CUPED_GMV
    y_control = gmv_data.query("treatment==0")[metric]
    X_cov_control = gmv_data.query("treatment==0")[["CUPED_X", "missing_CUPED"]]
    y_hat = sm.OLS(y_control, X_cov_control).fit().predict(gmv_data[["CUPED_X", "missing_CUPED"]])
    gmv_data["CUPED_GMV"] = gmv_data[metric] - y_hat
    # зафиксируем снижение дисперсии
    if optional:
        print(f"\nИсходная дисперсия = {gmv_data[metric].std():.8}")
        print(f"Дисперсия после cuped = {gmv_data.CUPED_GMV.std():.8}")
        print(f"Уменьшение на {1-gmv_data.CUPED_GMV.std()/gmv_data[metric].std():.3%}")
    gmv_test_observations = gmv_data.sample(frac=test_fraction)


    sd_gmv = gmv_test_observations.CUPED_GMV.std() # считаем по метрике CUPED_GMV
    est_gmv = gmv_test_observations[metric].mean() # считаем по дефолтной метрике, потому что размер эффекта не изменится

    ratio = (1-test_fraction) / (test_fraction) # пропорция контроль/тест
    lift_gmv = 0.05 # размер эффекта в %, для которого оцениваем мощность
    nobs_test = gmv_test_observations.shape[0] # количество наблюдений (юзеров) в тесте

    effect_size_gmv = tt_ind_solve_power(power=power, nobs1=nobs_test, alpha=alpha, ratio=ratio)
    mde_gmv = effect_size_gmv * sd_gmv/est_gmv
    if optional:
        print(f"sd_gmv = {sd_gmv}")
    mde_gmv_abs = effect_size_gmv * sd_gmv
    # print(f"est_gmv = {est_gmv}")
    # print(f"sd_gmv = {sd_gmv}")
    # в процентах
#     print(f"\nMDE равно {mde_gmv:.2%}")


    eff_size_gmv = lift_gmv * est_gmv/sd_gmv
    power_gmv = tt_ind_solve_power(effect_size=eff_size_gmv, nobs1=nobs_test, alpha=alpha, ratio=ratio)
    # мощность для эффекта в 0.75% (lift_aov)
    if optional:
        print(f"Мощность теста для эффекта в {lift_gmv:.4%} равна {power_gmv:.6}")


    nobs_gmv_test = tt_ind_solve_power(effect_size=eff_size_gmv, power=power, alpha=alpha, ratio=ratio)
    # ratio = nobs_control/nobs_test
    nobs_gmv_control = nobs_gmv_test*ratio
    # всего нужно наблюдений, наблюдений в тестовой группе, в контрольной
    if optional:
        print(f"Всего необходимо наблюдений: {round(nobs_gmv_control + nobs_gmv_test, 1):,}")
        print(f"Наблюдений в тестовой группе: {round(nobs_gmv_test, 1):,}")
        print(f"Наблюдений в контрольной группе: {round(nobs_gmv_control, 1):,}")
    if optional:
        print('---------------------------------------------------')
#     else:
#         print('-------------------------')

    red_dict = {
        "effect_size": effect_size_gmv,
        "mde": mde_gmv,
        "mde_abs": mde_gmv_abs
    }
    return red_dict

def filter_outliers(df, metric, k=9):
    df_mean = df[df[metric] > 0][metric].mean()
    df_std = df['gmv'].std()
    # print(f"df std before filter = {df_std:.2f}")
    rows_to_drop = df[metric] > df_mean + k * df_std

    df_filtered = df[~rows_to_drop]
    df_filtered_mean = df_filtered[metric].mean()
    df_filtered_std = df_filtered['gmv'].std()
    # print(f"df std after  filter = {df_filtered_std:.2f}")
    return df_filtered

def date_n_weeks_ago(n, param=False):
    # Get the current date
    if param:
        d=2
    else:
        d=1
    current_date = datetime.now() - timedelta(days=d)

    # Calculate the date n weeks ago
    date_n_weeks_ago = current_date - timedelta(weeks=n)

    # Format the date
    formatted_date = date_n_weeks_ago.strftime('%Y-%m-%d')

    return formatted_date


# Конверсия новичков апп в заказ

In [33]:
def collect_data(n_weeks) -> pd.DataFrame:
    q = f"""
        select
            anonymous_id,
            max(isNotNull(order_completed_at)) as ord_compl
        from new_app_funnel_table
        where order_flow='new'
            and b2b_flg in ('Представитель компании', 'Пользователь с флагом В2В')
            and observation_date between toDate('{date_n_weeks_ago(n_weeks)}') and toDate('{date_n_weeks_ago(0)}')
        group by anonymous_id
        """
    return get_query_clickhouse(q)

def collect_cuped_data(n_weeks) -> pd.DataFrame:
    q = f"""
        select
            anonymous_id,
            max(isNotNull(order_completed_at)) as ord_compl
        from new_app_funnel_table
        where order_flow='new'
            and b2b_flg in ('Представитель компании', 'Пользователь с флагом В2В')
            and observation_date between toDate('{date_n_weeks_ago(2*n_weeks, param=True)}') and toDate('{date_n_weeks_ago(n_weeks, param=True)}')   
        group by anonymous_id
        """
    return get_query_clickhouse(q)

In [34]:
for i in [3,4,5,6]:
    print(f'{i} недель:')
    exp_ios_data = collect_data(n_weeks=i)
    obs_ios_data = collect_cuped_data(n_weeks=i)
    for metric in ['ord_compl']:
        res = get_MDE(metric, exp_ios_data, obs_ios_data)
        effect_size = res['effect_size']
        mde_abs = res['mde_abs']
        mde = res['mde']
        print(f"mde = {mde:.2}")
        print(f"mde_abs = {mde_abs:.2}")
        print(f"effect_size = {effect_size:.2}")

3 недель:
mde = 0.17
mde_abs = 0.03
effect_size = 0.077
4 недель:
mde = 0.14
mde_abs = 0.028
effect_size = 0.069
5 недель:
mde = 0.12
mde_abs = 0.026
effect_size = 0.063
6 недель:
mde = 0.11
mde_abs = 0.025
effect_size = 0.059


# Конверсия mobile web в заказ на любой платформе

In [35]:
def collect_data(n_weeks) -> pd.DataFrame:
    q = f"""
        with t1 as (
            select
                user_id, max(shipped_at) as shipped_dt 
            from gp_rep.rep__bi_shipment
            where shipped_at between toDate('{date_n_weeks_ago(n_weeks)}') and toDate('{date_n_weeks_ago(0)}')
            group by user_id)

        , orders as (
        select 
            dictGet('analytics.spree_users_dict', 'uuid', toUInt64(user_id)) as uuid, shipped_dt 
        from t1)

        , users as (
        select user_id, min(observation_date) as dt
        from web_funnel
        where tenant='smbusiness'
            and device_type='mobile'
            and user_id<>''
            and isNotNull(login_at)
            and observation_date between toDate('{date_n_weeks_ago(n_weeks)}') and toDate('{date_n_weeks_ago(0)}')
        group by user_id)
            
        select user_id as anonymous_id, if(uuid<>'' and toDate(shipped_dt)>=toDate(dt), 1, 0) as conv_to_ord
        from users left join orders on users.user_id=orders.uuid
        """
    return get_query_clickhouse(q)

def collect_cuped_data(n_weeks) -> pd.DataFrame:
    q = f"""
        with t1 as (
            select
                user_id, max(shipped_at) as shipped_dt 
            from gp_rep.rep__bi_shipment
            where shipped_at between toDate('{date_n_weeks_ago(2*n_weeks, param=True)}') and toDate('{date_n_weeks_ago(n_weeks, param=True)}')   
            group by user_id)

        , orders as (
        select 
            dictGet('analytics.spree_users_dict', 'uuid', toUInt64(user_id)) as uuid, shipped_dt 
        from t1)

        , users as (
        select user_id, min(observation_date) as dt
        from web_funnel
        where tenant='smbusiness'
            and device_type='mobile'
            and user_id<>''
            and isNotNull(login_at)
            and observation_date between toDate('{date_n_weeks_ago(2*n_weeks, param=True)}') and toDate('{date_n_weeks_ago(n_weeks, param=True)}')   
        group by user_id)
            
        select user_id as anonymous_id, if(uuid<>'' and toDate(shipped_dt)>=toDate(dt), 1, 0) as conv_to_ord
        from users left join orders on users.user_id=orders.uuid
        """
    return get_query_clickhouse(q)

In [36]:
for i in [3,4,5,6]:
    print(f'{i} недель:')
    exp_ios_data = collect_data(n_weeks=i)
    obs_ios_data = collect_cuped_data(n_weeks=i)
    for metric in ['conv_to_ord']:
        res = get_MDE(metric, exp_ios_data, obs_ios_data)
        effect_size = res['effect_size']
        mde_abs = res['mde_abs']
        mde = res['mde']
        print(f"mde = {mde:.2}")
        print(f"mde_abs = {mde_abs:.2}")
        print(f"effect_size = {effect_size:.2}")

3 недель:
mde = 0.11
mde_abs = 0.047
effect_size = 0.099
4 недель:
mde = 0.09
mde_abs = 0.042
effect_size = 0.088
5 недель:
mde = 0.084
mde_abs = 0.039
effect_size = 0.08
6 недель:
mde = 0.076
mde_abs = 0.036
effect_size = 0.075


# GMV mobile web | app

In [37]:
def collect_data(n_weeks) -> pd.DataFrame:
    q = f"""
            select 
                user_id as anonymous_id,
                sum(gmv_goods_net_promo) as gmv
            from gp_rep.rep__bi_shipment
            where b2b_measure<>0
                and user_id <> 0
                and shipment_state='shipped' 
                and shipped_at between toDate('{date_n_weeks_ago(n_weeks)}') and toDate('{date_n_weeks_ago(0)}')
                and platform = 'web' and device_type='mobile' and tenant_id='smbusiness'
            group by anonymous_id
        """
    return get_query_clickhouse(q)

def collect_cuped_data(n_weeks) -> pd.DataFrame:
    q = f"""
        select 
            user_id as anonymous_id,
            sum(gmv_goods_net_promo) as gmv
        from gp_rep.rep__bi_shipment
        where b2b_measure<>0
            and user_id <> 0
            and shipment_state='shipped' 
            and shipped_at between toDate('{date_n_weeks_ago(2*n_weeks, param=True)}') and toDate('{date_n_weeks_ago(n_weeks, param=True)}')   
            and platform = 'web' and device_type='mobile' and tenant_id='smbusiness'
        group by anonymous_id
        """
    return get_query_clickhouse(q)

In [38]:
a = []
for i in [2,3,4,5,6]:
    print(f'{i} недель:')
    exp_ios_data = collect_data(n_weeks=i)
    exp_ios_data = filter_outliers(exp_ios_data, metric='gmv', k=9)
    obs_ios_data = collect_cuped_data(n_weeks=i)

    print(f"Уникальных пользователей за {i} недель {exp_ios_data['anonymous_id'].nunique()}")

    for _ in range(100): 
        mde_array=[]
        for metric in ['gmv']:
            res = get_MDE(metric, exp_ios_data, obs_ios_data)
            mde = res['mde']
            mde_array.append(mde)
    a.append(np.mean(mde_array))
a

2 недель:
Уникальных пользователей за 2 недель 532
3 недель:
Уникальных пользователей за 3 недель 702
4 недель:
Уникальных пользователей за 4 недель 820
5 недель:
Уникальных пользователей за 5 недель 940
6 недель:
Уникальных пользователей за 6 недель 1072


[0.3025503791639217,
 0.2434235254476935,
 0.21000413547069288,
 0.2638115254906051,
 0.25571235091074723]

# APP

In [23]:
def collect_data(n_weeks) -> pd.DataFrame:
    q = f"""
            select 
                user_id as anonymous_id,
                sum(gmv_goods_net_promo) as gmv
            from gp_rep.rep__bi_shipment
            where b2b_measure<>0
                and user_id <> 0
                and shipment_state='shipped' 
                and shipped_at between toDate('{date_n_weeks_ago(n_weeks)}') and toDate('{date_n_weeks_ago(0)}')
                and platform='app'
            group by anonymous_id
        """
    return get_query_clickhouse(q)

def collect_cuped_data(n_weeks) -> pd.DataFrame:
    q = f"""
        select 
            user_id as anonymous_id,
            sum(gmv_goods_net_promo) as gmv
        from gp_rep.rep__bi_shipment
        where b2b_measure<>0
            and user_id <> 0
            and shipment_state='shipped' 
            and shipped_at between toDate('{date_n_weeks_ago(2*n_weeks, param=True)}') and toDate('{date_n_weeks_ago(n_weeks, param=True)}')   
            and platform='app'
        group by anonymous_id
        """
    return get_query_clickhouse(q)

In [24]:
a = []
for i in [2,3,4,5,6]:
    print(f'{i} недель:')
    exp_ios_data = collect_data(n_weeks=i)
    exp_ios_data = filter_outliers(exp_ios_data, metric='gmv', k=9)
    obs_ios_data = collect_cuped_data(n_weeks=i)

    print(f"Уникальных пользователей за {i} недель {exp_ios_data['anonymous_id'].nunique()}")

    for _ in range(100): 
        mde_array=[]
        for metric in ['gmv']:
            res = get_MDE(metric, exp_ios_data, obs_ios_data)
            mde = res['mde']
            mde_array.append(mde)
    a.append(np.mean(mde_array))
a

2 недель:
Уникальных пользователей за 2 недель 37729
3 недель:
Уникальных пользователей за 3 недель 46018
4 недель:
Уникальных пользователей за 4 недель 52373
5 недель:
Уникальных пользователей за 5 недель 58402
6 недель:
Уникальных пользователей за 6 недель 63803


[0.03288366198961904,
 0.032823740707284034,
 0.03148108875762498,
 0.02887340200099975,
 0.02780745222225427]

# Конверсия из чекаута в b2b заказ на апп

In [39]:
def collect_data(n_weeks) -> pd.DataFrame:
    q = f"""
        select 
            anonymous_id,
            max(if(isNotNull(order_completed_at) 
            and context_order_id in (select order_number from gp_rep.rep__bi_shipment where b2b_measure<>0 and shipped_at>'2024-10-10'),
            1, 0)) as b2b_ord_cmpl
        from new_app_funnel_table
        where isNotNull(checkout_loaded_at)
        and observation_date between toDate('{date_n_weeks_ago(n_weeks)}') and toDate('{date_n_weeks_ago(0)}')
        group by anonymous_id
        """
    return get_query_clickhouse(q)

def collect_cuped_data(n_weeks) -> pd.DataFrame:
    q = f"""
        select 
            anonymous_id,
            max(if(isNotNull(order_completed_at) 
            and context_order_id in (select order_number from gp_rep.rep__bi_shipment where b2b_measure<>0 and shipped_at>'2024-10-10'),
            1, 0)) as b2b_ord_cmpl
        from new_app_funnel_table
        where isNotNull(checkout_loaded_at)
        and observation_date between toDate('{date_n_weeks_ago(2*n_weeks, param=True)}') and toDate('{date_n_weeks_ago(n_weeks, param=True)}')   
        group by anonymous_id
        """
    return get_query_clickhouse(q)

In [40]:
for i in [3,4]:
    print(f'{i} недель:')
    exp_ios_data = collect_data(n_weeks=i)
    obs_ios_data = collect_cuped_data(n_weeks=i)
    for metric in ['b2b_ord_cmpl']:
        res = get_MDE(metric, exp_ios_data, obs_ios_data)
        effect_size = res['effect_size']
        mde_abs = res['mde_abs']
        mde = res['mde']
        print(f"mde = {mde:.2}")
        print(f"mde_abs = {mde_abs:.2}")
        print(f"effect_size = {effect_size:.2}")

3 недель:
mde = 0.027
mde_abs = 0.00054
effect_size = 0.0038
4 недель:
mde = 0.027
mde_abs = 0.00046
effect_size = 0.0035


In [41]:
res

{'effect_size': 0.003513743500419804,
 'mde': 0.026603156604382356,
 'mde_abs': 0.0004582594982632454}